In [1]:
# !pip install telepot
# !pip install telepot --upgrade
# !pip install pyTelegramBotAPI
# !pip install icalendar
# !pip install django

In [2]:
from telepot.loop import MessageLoop
import telepot

In [3]:
from libs.googleCalendar import createCalendar, processCalendar
from libs.feisBot import feisCallback
from libs.botUtils import createMarkup

In [4]:
import codecs
def addNews(news):
    print 'addNews', news
    nfile = codecs.open('News.txt','a',encoding='utf8')     
    nfile.write('\n')
    nfile.write(news)
    nfile.close()
    
def newsProcess(bot, chat_id, data, sender):
    if data[0] == '/news':
        nfile = codecs.open('News.txt','r',encoding='utf8')     
        bot.sendMessage(chat_id, u''.join(nfile))
        return
    if sender not in ('djack239', 'Draco_vad'):
        if data[0] == '/addNews':
            print 'badSender [{}]'.format(sender)
        return
    if data[0] == '/addNews':
        addNews(u' '.join(data[1:]))
    elif data[0] == '/clearNews':
        nfile = codecs.open('News.txt','w',encoding='utf8')
        nfile.close()


In [5]:
def handleImpl(msg):
    content_type, chat_type, chat_id = telepot.glance(msg)
    request = msg['text'].split(' ')
    if request[0] == '/schedule':
        markup = createCalendar()
        bot.sendMessage(chat_id, 'Выберите день', reply_markup=markup)
    elif request[0] == '/feis':
        keys = [(('Стэйдж листы по школам', 'feiss'),),
                (('Статистика по фэшу', 'feisStat'),)]
        markup = createMarkup(keys)
        bot.sendMessage(chat_id, 'Что вы хотите узнать', reply_markup=markup)
    newsProcess(bot, chat_id, request, msg['from']['username'] )

def callbackImpl(msg):
    query_id, chat_id, query_data = telepot.glance(msg, flavor='callback_query')
    data = query_data.split(' ')
    processCalendar(bot, chat_id, data)
    feisCallback(bot, chat_id, data)


def choseImpl(msg):
    print 'choseImpl', msg

In [6]:
def write(fileName, t, msg):
    with open('Logs/{}.txt'.format(fileName), 'a') as logFile:
        logFile.write(t)
        logFile.write('\n')
        logFile.write(str(msg))
        logFile.write('\n')
        logFile.write('\n')


In [7]:
def handle(msg):
    write('logT', 'handle', msg)
    return handleImpl(msg)
def callback(msg):
    write('logT', 'callback', msg)
    return callbackImpl(msg)
def chose(msg):
    write('logT', 'chose', msg)
    return choseImpl(msg)

token = next(open('token.txt'))
bot = telepot.Bot(token)
MessageLoop(bot, {'chat': handle,
                  'callback_query': callback,
                  'chosen_inline_result': chose}).run_as_thread()

In [8]:
def handleImplF(msg):
    content_type, chat_type, chat_id = telepot.glance(msg)
    request = msg['text'].split(' ')
    if request[0] == '/feis':
        keys = [(('Стэйдж листы по школам', 'feiss'),),
                (('Статистика по фэшу', 'feisStat'),)]
        markup = createMarkup(keys)
        botf.sendMessage(chat_id, 'Что вы хотите узнать', reply_markup=markup)

def callbackImplF(msg):
    query_id, chat_id, query_data = telepot.glance(msg, flavor='callback_query')
    data = query_data.split(' ')
    feisCallback(botf, chat_id, data)


def choseImplF(msg):
    print 'choseImpl', msg


In [9]:
def handleF(msg):
    write('logF', 'handle', msg)
    return handleImplF(msg)
def callbackF(msg):
    write('logF', 'callback', msg)
    return callbackImplF(msg)
def choseF(msg):
    write('logF', 'chose', msg)
    return choseImplF(msg)

token = next(open('feisToken.txt'))
botf = telepot.Bot(token)
MessageLoop(botf, {'chat': handleF,
                  'callback_query': callbackF,
                  'chosen_inline_result': choseF}).run_as_thread()